In [62]:
import cv2
import numpy as np
import scipy.signal as sp
import dlib

import matplotlib.pyplot as plt
from matplotlib.pyplot import imshow
from matplotlib.pyplot import Polygon, Rectangle

%matplotlib inline

In [178]:
#
# Initialize dlib modules
#
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("./shape_predictor_68_face_landmarks.dat")

In [180]:
#
# Parameters
#
SQUARE_SIZE = 5
HEIGHT_SCALE = 16
HEIGHT_WIDTH = 1
BLACK_OFFSET = HEIGHT_WIDTH*HEIGHT_SCALE + SQUARE_SIZE

#
# Colors
#
clr_base_tile = 70
clr_tile_border = 25
clr_south_wall = 40
clr_west_wall = 50

#
# Objects
#
obj_base_tile = np.zeros((SQUARE_SIZE, SQUARE_SIZE))
obj_base_tile.fill(clr_base_tile)
obj_base_tile[0, :] = clr_tile_border
obj_base_tile[:, SQUARE_SIZE - 1] = clr_tile_border

In [188]:
cap = cv2.VideoCapture(0)

while True:
    _, frame = cap.read()
    
    # convert to grayscale
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    # initialize the face recognizer (default face haar cascade)
    # download the XML file from here: https://github.com/opencv/opencv/tree/master/data/haarcascades
    #v1face_cascade = cv2.CascadeClassifier("cascades/haarcascade_frontalface_alt.xml")
    
    # detect all the faces in the image
    #v1faces = face_cascade.detectMultiScale(gray)
    
    faces = detector(gray, 1)

    for k, d in enumerate(faces):
        # Get the landmarks/parts for the face in box d.
        shape = predictor(gray, d)
        x1 = shape.part(0).x
        y1 = shape.part(0).y
        x2 = shape.part(16).x
        y2 = shape.part(16).y
        A = y2 - y1
        B = -(x2 - x1)
        C = -A*x1 - B*y1
        M = np.sqrt(A**2 + B**2)    
        AA = A / M
        BB = B / M
        CC = C / M

        coords = np.zeros((32, 2)).astype(np.int32)

        for ii in range(17):
            coords[ii, :] = (shape.part(ii).x, shape.part(ii).y)

        for ii in range(15, 0, -1):
            px = shape.part(ii).x
            py = shape.part(ii).y

            D = AA * px + BB * py + CC
            pxx = int(px - 2*AA*D)
            pyy = int(py - 2*BB*D)

            coords[32 - ii, :] = (pxx, pyy)

        # Face box
        XUL = min(coords, key = lambda x: x[0])[0] - 20
        YUL = min(coords, key = lambda x: x[1])[1] - 20
        XLR = max(coords, key = lambda x: x[0])[0] + 20
        YLR = max(coords, key = lambda x: x[1])[1] + 20

        facebox = Rectangle((XUL, YUL), width=(XLR - XUL), height=(YLR - YUL), fill=False)
        face = gray[YUL:YLR, XUL:XLR]

    coords[:, 0] = coords[:, 0] - XUL
    coords[:, 1] = coords[:, 1] - YUL
    
    stencil = np.zeros(face.shape).astype(face.dtype)
    cv2.fillPoly(stencil, [coords], 255)
    result = np.bitwise_and(face, stencil)
    
    #
    # Convert input black and white image to small picture and mean convolve
    #
    filter = np.zeros((SQUARE_SIZE, SQUARE_SIZE))
    filter.fill(1.0 / SQUARE_SIZE**2)

    small_img = sp.convolve2d(result, filter[::-1, ::-1], mode="valid")[::SQUARE_SIZE, ::SQUARE_SIZE]

    #
    # Scale the image of tiles
    #
    scaled_small_img = small_img - small_img.min()
    scale = np.linspace(0, scaled_small_img.max(), HEIGHT_SCALE + 1)
    scaled_small_img = np.digitize(scaled_small_img, scale) - 1

    #
    # Main body of the effect
    #

    # This one makes the black border!
    final_img = np.zeros((face.shape[0] + BLACK_OFFSET, face.shape[1] + BLACK_OFFSET)).astype(np.uint8)

    # Actually fills the image with tiles and height
    for ii in range(scaled_small_img.shape[0]):
        for jj in range(scaled_small_img.shape[1] - 1, -1, -1):
            height_value = int(scaled_small_img[ii, jj])

            if (height_value > 0):
                left_lower_row = (ii + 1)*SQUARE_SIZE - 1 + BLACK_OFFSET
                left_lower_col = jj*SQUARE_SIZE

                for kk in range(0, height_value*HEIGHT_WIDTH):
                    final_img[left_lower_row - kk, (left_lower_col + kk):(left_lower_col + SQUARE_SIZE + kk)] = clr_south_wall
                    final_img[(left_lower_row - SQUARE_SIZE - kk + BLACK_OFFSET):(left_lower_row - kk - 1 + BLACK_OFFSET), left_lower_col + kk] = clr_west_wall

                final_img[(left_lower_row - SQUARE_SIZE - kk):(left_lower_row - kk), 
                          (left_lower_col + kk):(left_lower_col + SQUARE_SIZE + kk)] = obj_base_tile

    
    # Sow everything
    cv2.imshow("image", gray)
    cv2.imshow("face", face)
    cv2.imshow("result", result)
    cv2.imshow("Effect", final_img)
    
    if (cv2.waitKey(1) & 0xFF == ord("q")):
        break

cap.release()
cv2.destroyAllWindows()

In [183]:
cap.release()
cv2.destroyAllWindows()